In [1]:
import scanpy as sc
import numpy as np
import pandas as pd
import streamlit as st

In [2]:
adata = sc.read('/Users/derekebowman/Coding Projects/PhD_Bowman/Data/taPVAT_combined_annotated_with_immune_fibro_ecs_all_genes_unnormalized.h5ad')



In [3]:
sc.pp.filter_genes(adata, min_counts = 10)
sc.pp.filter_genes(adata, min_cells = 20)
sc.pp.filter_cells(adata, min_genes=50)
sc.pp.filter_cells(adata, min_counts=100)
sc.pp.normalize_total(adata) 

In [4]:
adata

AnnData object with n_obs × n_vars = 71813 × 17797
    obs: 'sample_id', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'scDblFinder_score', 'scDblFinder_class', 'Sample', 'tissue', 'diet', 'time', 'sex', 'batch', 'sample_type', 'rat_id', 'weight_3W', 'weight_10W', 'weight_17W', 'weight_24W', 'MAP_7W', 'systolic_bp_7W', 'MAP_9W', 'systolic_bp_9W', 'MAP_16W', 'systolic_bp_16W', 'MAP_24W', 'systolic_bp_24W', 'doublet', 'celltype', 'celltype_broad', 'n_counts'
    var: 'gene_ids', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts', 'n_cells', 'n_counts'
    layers: 'counts', 'raw_counts', 'soupX_counts'

In [5]:
def TimeDietSex(row):
    if row['time'] == '8W' and row['diet'] == 'Control' and row['sex'] == 'F':
        return 'F_8W_Control'
    
    elif row['time'] == '8W' and row['diet'] == 'HF' and row['sex'] == 'F':
        return 'F_8W_HF'
    
    elif row['time'] == '24W' and row['diet'] == 'HF' and row['sex'] == 'F':
        return 'F_24W_HF'
    
    elif row['time'] == '24W' and row['diet'] == 'Control' and row['sex'] == 'F':
        return 'F_24W_Control'
    
    if row['time'] == '8W' and row['diet'] == 'Control' and row['sex'] == 'M':
        return 'M_8W_Control'
    
    elif row['time'] == '8W' and row['diet'] == 'HF' and row['sex'] == 'M':
        return 'M_8W_HF'
    
    elif row['time'] == '24W' and row['diet'] == 'HF' and row['sex'] == 'M':
        return 'M_24W_HF'
    
    elif row['time'] == '24W' and row['diet'] == 'Control' and row['sex'] == 'M':
        return 'M_24W_Control'
    else:
        return 'Error'
    
    
adata.obs['time_diet_sex'] = adata.obs.apply(TimeDietSex, axis=1)

In [6]:
adata.obs['time_diet_sex']

AAACGAACACGACGAA-1-0     M_8W_Control
AAACGCTGTATGAGGC-1-0     M_8W_Control
AAAGAACAGGGCAGGA-1-0     M_8W_Control
AAAGAACCAAATGCTC-1-0     M_8W_Control
AAAGAACGTTCGAACT-1-0     M_8W_Control
                             ...     
TTTGTTGCACCTGCGA-1-24         F_8W_HF
TTTGTTGCACTATGTG-1-24         F_8W_HF
TTTGTTGGTAGATGTA-1-24         F_8W_HF
TTTGTTGGTTCATCTT-1-24         F_8W_HF
TTTGTTGTCTCCAATT-1-24         F_8W_HF
Name: time_diet_sex, Length: 71813, dtype: object

In [7]:
dense_matrix = adata.X.toarray()

In [8]:
df = pd.DataFrame(dense_matrix, 
                  columns=adata.var.index, 
                  index=pd.MultiIndex.from_arrays([adata.obs['celltype_broad'], adata.obs['time_diet_sex']], 
                                                  names=['celltype_broad', 'time_diet_sex']))


In [9]:
df.reset_index(inplace=True)

In [10]:
df

,celltype_broad,time_diet_sex,3110082J24Rik,4930455H04Rik,6330411D24Rik,A1cf,A3galt2,AABR07000398.1,AABR07000544.1,AABR07001512.1,...,Zyg11a,Zyg11b,Zyx,Zzef1,Zzz3,hist1h2ail2,mrpl11,mrpl24,mrpl9,rnf141
0,Fibroblasts,M_8W_Control,0.000000,0.0,0.0,0.0,0.0,2.421107,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0000
1,Adipocytes,M_8W_Control,0.000000,0.0,0.0,0.0,0.0,5.463584,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0000
2,Immune_Cells,M_8W_Control,0.000000,0.0,0.0,0.0,0.0,5.216336,0.0,0.0,...,0.0,0.000000,0.0,2.608168,0.000000,0.0,0.0,0.0,0.0,0.0000
3,Adipocytes,M_8W_Control,0.000000,0.0,0.0,0.0,0.0,10.533432,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0000
4,Adipocytes,M_8W_Control,0.558101,0.0,0.0,0.0,0.0,5.581011,0.0,0.0,...,0.0,0.000000,0.0,0.558101,0.558101,0.0,0.0,0.0,0.0,0.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71808,Adipocytes,F_8W_HF,0.000000,0.0,0.0,0.0,0.0,2.762666,0.0,0.0,...,0.0,0.920889,0.0,1.841777,0.000000,0.0,0.0,0.0,0.0,0.0000
71809,Adipocytes,F_8W_HF,0.000000,0.0,0.0,0.0,0.0,2.131710,0.0,0.0,...,0.0,0.000000,0.0,0.355285,0.000000,0.0,0.0,0.0,0.0,0.0000
71810,Adipocytes,F_8W_HF,0.000000,0.0,0.0,0.0,0.0,5.444700,0.0,0.0,...,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,1.8149
71811,Pericytes,F_8W_HF,0.000000,0.0,0.0,0.0,0.0,1.459543,0.0,0.0,...,0.0,0.000000,0.0,1.459543,0.000000,0.0,0.0,0.0,0.0,0.0000


In [11]:
genes_time_diet_sex_described = df.groupby(['time_diet_sex', 'celltype_broad']).describe()

In [12]:
genes_time_diet_sex_described

3110082J24Rik                                \
                                        count      mean       std  min  25%   
time_diet_sex celltype_broad                                                  
F_24W_Control Adipocytes               4387.0  0.010370  0.104041  0.0  0.0   
              Endothelial_Cells        1790.0  0.000841  0.035574  0.0  0.0   
              Fibroblasts               735.0  0.001433  0.038859  0.0  0.0   
              Immune_Cells              239.0  0.000000  0.000000  0.0  0.0   
              Mesothelial_Cells          56.0  0.000000  0.000000  0.0  0.0   
...                                       ...       ...       ...  ...  ...   
M_8W_HF       Immune_Cells              577.0  0.001895  0.032883  0.0  0.0   
              Mesothelial_Cells         193.0  0.000000  0.000000  0.0  0.0   
              Neuronal_Cells             22.0  0.000000  0.000000  0.0  0.0   
              Pericytes                 426.0  0.000000  0.000000  0.0  0.0   
              SMCs                        8.0  0.000000  0.000000  0.0  0.0   

                                                    4930455H04Rik            \
                                 50%  75%       max         count      mean   
time_diet_sex celltype_broad                                                  
F_24W_Control Adipocytes         0.0  0.0  2.394124        4387.0  0.000867   
              Endothelial_Cells  0.0  0.0  1.505096        1790.0  0.003392   
              Fibroblasts        0.0  0.0  1.053500         735.0  0.003869   
              Immune_Cells       0.0  0.0  0.000000         239.0  0.000000   
              Mesothelial_Cells  0.0  0.0  0.000000          56.0  0.000000   
...                              ...  ...       ...           ...       ...   
M_8W_HF       Immune_Cells       0.0  0.0  0.663391         577.0  0.000000   
              Mesothelial_Cells  0.0  0.0  0.000000         193.0  0.009580   
              Neuronal_Cells     0.0  0.0  0.000000          22.0  0.000000   
              Pericytes          0.0  0.0  0.000000         426.0  0.000000   
              SMCs               0.0  0.0  0.000000           8.0  0.000000   

                                 ... mrpl9            rnf141            \
                                 ...   75%       max   count      mean   
time_diet_sex celltype_broad     ...                                     
F_24W_Control Adipocytes         ...   0.0  4.597276  4387.0  0.059599   
              Endothelial_Cells  ...   0.0  4.416822  1790.0  0.090911   
              Fibroblasts        ...   0.0  6.761087   735.0  0.090517   
              Immune_Cells       ...   0.0  3.444606   239.0  0.105473   
              Mesothelial_Cells  ...   0.0  0.000000    56.0  0.011134   
...                              ...   ...       ...     ...       ...   
M_8W_HF       Immune_Cells       ...   0.0  4.367837   577.0  0.124296   
              Mesothelial_Cells  ...   0.0  3.635385   193.0  0.117893   
              Neuronal_Cells     ...   0.0  0.000000    22.0  0.000000   
              Pericytes          ...   0.0  4.526820   426.0  0.078814   
              SMCs               ...   0.0  0.000000     8.0  0.000000   

                                                                         
                                      std  min  25%  50%  75%       max  
time_diet_sex celltype_broad                                             
F_24W_Control Adipocytes         0.248151  0.0  0.0  0.0  0.0  4.492395  
              Endothelial_Cells  0.441249  0.0  0.0  0.0  0.0  6.098065  
              Fibroblasts        0.432965  0.0  0.0  0.0  0.0  4.688492  
              Immune_Cells       0.505976  0.0  0.0  0.0  0.0  3.861111  
              Mesothelial_Cells  0.058540  0.0  0.0  0.0  0.0  0.334560  
...                                   ...  ...  ...  ...  ...       ...  
M_8W_HF       Immune_Cells       0.677816  0.0  0.0  0.0  0.0  9.528226  
              Mesothelial_Cells  0.560333

In [18]:
genes_time_diet_sex_described.to_csv('genes_time_diet_sex_described.csv')

In [31]:
df = pd.read_csv('genes_time_diet_sex_described.csv', header=[0, 1], index_col=[0, 1])

In [34]:
celltypes = df.index.get_level_values('celltype_broad').unique()
time_diet_sex_groups = df.index.get_level_values('time_diet_sex').unique()

In [36]:
time_diet_sex_groups

Index(['F_24W_Control', 'F_24W_HF', 'F_8W_Control', 'F_8W_HF', 'M_24W_Control',
       'M_24W_HF', 'M_8W_Control', 'M_8W_HF'],
      dtype='object', name='time_diet_sex')

In [37]:
gene_options = df.columns.tolist()

In [42]:
# Assuming your DataFrame is named df
gene_names = df.columns.get_level_values(0).unique().tolist()

# Now, gene_names contains a list of unique gene names
print(gene_names)


['3110082J24Rik', '4930455H04Rik', '6330411D24Rik', 'A1cf', 'A3galt2', 'AABR07000398.1', 'AABR07000544.1', 'AABR07001512.1', 'AABR07001592.2', 'AABR07001902.1', 'AABR07001942.1', 'AABR07002093.2', 'AABR07002467.1', 'AABR07002627.1', 'AABR07002783.1', 'AABR07002969.1', 'AABR07004112.1', 'AABR07004232.1', 'AABR07004891.1', 'AABR07005588.1', 'AABR07005806.1', 'AABR07006049.1', 'AABR07006111.1', 'AABR07006283.1', 'AABR07006685.1', 'AABR07006688.2', 'AABR07006691.1', 'AABR07007032.1', 'AABR07007121.1', 'AABR07007905.2', 'AABR07008066.2', 'AABR07008309.1', 'AABR07008608.1', 'AABR07008681.1', 'AABR07008940.1', 'AABR07009105.1', 'AABR07009260.1', 'AABR07009572.1', 'AABR07009634.2', 'AABR07011733.1', 'AABR07012054.1', 'AABR07012592.1', 'AABR07013057.1', 'AABR07013165.1', 'AABR07013425.1', 'AABR07014306.2', 'AABR07015057.1', 'AABR07015346.1', 'AABR07015507.1', 'AABR07015559.1', 'AABR07015941.1', 'AABR07017236.1', 'AABR07017236.2', 'AABR07017902.1', 'AABR07017917.1', 'AABR07018124.1', 'AABR070181

In [47]:
 df = pd.read_csv('genes_time_diet_sex_described.csv', header=[0, 1], index_col=[0, 1])

In [48]:
df

3110082J24Rik                                \
                                        count      mean       std  min  25%   
time_diet_sex celltype_broad                                                  
F_24W_Control Adipocytes               4387.0  0.010370  0.104041  0.0  0.0   
              Endothelial_Cells        1790.0  0.000841  0.035574  0.0  0.0   
              Fibroblasts               735.0  0.001433  0.038859  0.0  0.0   
              Immune_Cells              239.0  0.000000  0.000000  0.0  0.0   
              Mesothelial_Cells          56.0  0.000000  0.000000  0.0  0.0   
...                                       ...       ...       ...  ...  ...   
M_8W_HF       Immune_Cells              577.0  0.001895  0.032883  0.0  0.0   
              Mesothelial_Cells         193.0  0.000000  0.000000  0.0  0.0   
              Neuronal_Cells             22.0  0.000000  0.000000  0.0  0.0   
              Pericytes                 426.0  0.000000  0.000000  0.0  0.0   
              SMCs                        8.0  0.000000  0.000000  0.0  0.0   

                                                    4930455H04Rik            \
                                 50%  75%       max         count      mean   
time_diet_sex celltype_broad                                                  
F_24W_Control Adipocytes         0.0  0.0  2.394124        4387.0  0.000867   
              Endothelial_Cells  0.0  0.0  1.505096        1790.0  0.003392   
              Fibroblasts        0.0  0.0  1.053500         735.0  0.003869   
              Immune_Cells       0.0  0.0  0.000000         239.0  0.000000   
              Mesothelial_Cells  0.0  0.0  0.000000          56.0  0.000000   
...                              ...  ...       ...           ...       ...   
M_8W_HF       Immune_Cells       0.0  0.0  0.663391         577.0  0.000000   
              Mesothelial_Cells  0.0  0.0  0.000000         193.0  0.009580   
              Neuronal_Cells     0.0  0.0  0.000000          22.0  0.000000   
              Pericytes          0.0  0.0  0.000000         426.0  0.000000   
              SMCs               0.0  0.0  0.000000           8.0  0.000000   

                                 ... mrpl9            rnf141            \
                                 ...   75%       max   count      mean   
time_diet_sex celltype_broad     ...                                     
F_24W_Control Adipocytes         ...   0.0  4.597276  4387.0  0.059599   
              Endothelial_Cells  ...   0.0  4.416822  1790.0  0.090911   
              Fibroblasts        ...   0.0  6.761087   735.0  0.090517   
              Immune_Cells       ...   0.0  3.444606   239.0  0.105473   
              Mesothelial_Cells  ...   0.0  0.000000    56.0  0.011134   
...                              ...   ...       ...     ...       ...   
M_8W_HF       Immune_Cells       ...   0.0  4.367837   577.0  0.124296   
              Mesothelial_Cells  ...   0.0  3.635385   193.0  0.117893   
              Neuronal_Cells     ...   0.0  0.000000    22.0  0.000000   
              Pericytes          ...   0.0  4.526820   426.0  0.078814   
              SMCs               ...   0.0  0.000000     8.0  0.000000   

                                                                         
                                      std  min  25%  50%  75%       max  
time_diet_sex celltype_broad                                             
F_24W_Control Adipocytes         0.248151  0.0  0.0  0.0  0.0  4.492395  
              Endothelial_Cells  0.441249  0.0  0.0  0.0  0.0  6.098065  
              Fibroblasts        0.432965  0.0  0.0  0.0  0.0  4.688492  
              Immune_Cells       0.505976  0.0  0.0  0.0  0.0  3.861111  
              Mesothelial_Cells  0.058540  0.0  0.0  0.0  0.0  0.334560  
...                                   ...  ...  ...  ...  ...       ...  
M_8W_HF       Immune_Cells       0.677816  0.0  0.0  0.0  0.0  9.528226  
              Mesothelial_Cells  0.560333

In [60]:
df

3110082J24Rik                                \
                                        count      mean       std  min  25%   
time_diet_sex celltype_broad                                                  
F_24W_Control Adipocytes               4387.0  0.010370  0.104041  0.0  0.0   
              Endothelial_Cells        1790.0  0.000841  0.035574  0.0  0.0   
              Fibroblasts               735.0  0.001433  0.038859  0.0  0.0   
              Immune_Cells              239.0  0.000000  0.000000  0.0  0.0   
              Mesothelial_Cells          56.0  0.000000  0.000000  0.0  0.0   
...                                       ...       ...       ...  ...  ...   
M_8W_HF       Immune_Cells              577.0  0.001895  0.032883  0.0  0.0   
              Mesothelial_Cells         193.0  0.000000  0.000000  0.0  0.0   
              Neuronal_Cells             22.0  0.000000  0.000000  0.0  0.0   
              Pericytes                 426.0  0.000000  0.000000  0.0  0.0   
              SMCs                        8.0  0.000000  0.000000  0.0  0.0   

                                                    4930455H04Rik            \
                                 50%  75%       max         count      mean   
time_diet_sex celltype_broad                                                  
F_24W_Control Adipocytes         0.0  0.0  2.394124        4387.0  0.000867   
              Endothelial_Cells  0.0  0.0  1.505096        1790.0  0.003392   
              Fibroblasts        0.0  0.0  1.053500         735.0  0.003869   
              Immune_Cells       0.0  0.0  0.000000         239.0  0.000000   
              Mesothelial_Cells  0.0  0.0  0.000000          56.0  0.000000   
...                              ...  ...       ...           ...       ...   
M_8W_HF       Immune_Cells       0.0  0.0  0.663391         577.0  0.000000   
              Mesothelial_Cells  0.0  0.0  0.000000         193.0  0.009580   
              Neuronal_Cells     0.0  0.0  0.000000          22.0  0.000000   
              Pericytes          0.0  0.0  0.000000         426.0  0.000000   
              SMCs               0.0  0.0  0.000000           8.0  0.000000   

                                 ... mrpl9            rnf141            \
                                 ...   75%       max   count      mean   
time_diet_sex celltype_broad     ...                                     
F_24W_Control Adipocytes         ...   0.0  4.597276  4387.0  0.059599   
              Endothelial_Cells  ...   0.0  4.416822  1790.0  0.090911   
              Fibroblasts        ...   0.0  6.761087   735.0  0.090517   
              Immune_Cells       ...   0.0  3.444606   239.0  0.105473   
              Mesothelial_Cells  ...   0.0  0.000000    56.0  0.011134   
...                              ...   ...       ...     ...       ...   
M_8W_HF       Immune_Cells       ...   0.0  4.367837   577.0  0.124296   
              Mesothelial_Cells  ...   0.0  3.635385   193.0  0.117893   
              Neuronal_Cells     ...   0.0  0.000000    22.0  0.000000   
              Pericytes          ...   0.0  4.526820   426.0  0.078814   
              SMCs               ...   0.0  0.000000     8.0  0.000000   

                                                                         
                                      std  min  25%  50%  75%       max  
time_diet_sex celltype_broad                                             
F_24W_Control Adipocytes         0.248151  0.0  0.0  0.0  0.0  4.492395  
              Endothelial_Cells  0.441249  0.0  0.0  0.0  0.0  6.098065  
              Fibroblasts        0.432965  0.0  0.0  0.0  0.0  4.688492  
              Immune_Cells       0.505976  0.0  0.0  0.0  0.0  3.861111  
              Mesothelial_Cells  0.058540  0.0  0.0  0.0  0.0  0.334560  
...                                   ...  ...  ...  ...  ...       ...  
M_8W_HF       Immune_Cells       0.677816  0.0  0.0  0.0  0.0  9.528226  
              Mesothelial_Cells  0.560333